**Experimenting with different timezone information providers**
* datetime.timezone - represents a fixed offset from UTC
* pytz - allows handling of daylight saving time transitions
* dateutil.tz - provides time zone implementations for working with fixed offsets, local timezones
* zoneinfo - supports fixed and dynamic (with DST) time zones

In [13]:
#using python's datetime library (https://docs.python.org/3/library/datetime.html)
from datetime import datetime, timedelta
#this function returns the time from now until 5pm the next day
def time_until_5_tmrw(currtime = None):
    timenow = datetime.now()
    if(currtime!=None): timenow = currtime
    timetmrw = timenow + timedelta(days=1) #add one day
    tmrw5 = datetime(timetmrw.year, timetmrw.month, timetmrw.day, 17, 0, 0)
    return (tmrw5 - timenow).total_seconds() / 3600

#using datetime.timezone
from datetime import timezone
#this function returns the time from now until 5pm the next day
def time_until_5_tmrw_timezone(currtime = None):
    if(currtime==None): currtime = datetime.now(timezone.utc)
    timetmrw = currtime + timedelta(days=1) #add one day
    tmrw5 = datetime(timetmrw.year, timetmrw.month, timetmrw.day, 17, 0, 0, tzinfo=currtime.tzinfo)
    return (tmrw5 - currtime).total_seconds() / 3600

#using pytz
import pytz
#this function returns the time from now until 5pm the next day
def time_until_5_tmrw_pytz(currtime = None):
    if(currtime==None): currtime = datetime.now(pytz.UTC)
    timetmrw = currtime + timedelta(days=1) #add one day
    tmrw5 = currtime.astimezone(pytz.UTC).replace(timetmrw.year, timetmrw.month, timetmrw.day, 17, 0, 0)
    #currtime.astimezone(pytz.UTC) - converts currtime to UTC
    #.replace(...) - replaces the date and time components of the datetime object
    #incorrect b/c tmrw5 will be in UTC regardless of the og timezone of currtime
    #but will probably correctly handle daylight saving time
    return (tmrw5 - currtime).total_seconds() / 3600

#using dateutil.tz
from dateutil import tz
#this function returns the time from now until 5pm the next day
def time_until_5_tmrw_tz(currtime = None):
    if(currtime==None): currtime = datetime.now(tz.UTC)
    timetmrw = currtime + timedelta(days=1) #add one day
    tmrw5 = currtime.astimezone(tz.UTC).replace(timetmrw.year, timetmrw.month, timetmrw.day, 17, 0, 0)
    return (tmrw5 - currtime).total_seconds() / 3600

#using zoneinfo
from zoneinfo import ZoneInfo
#this function returns the time from now until 5pm the next day
def time_until_5_tmrw_zoneinfo(currtime = None):
    if(currtime==None): currtime = datetime.now(ZoneInfo("UTC"))
    timetmrw = currtime + timedelta(days=1) #add one day
    tmrw5 = currtime.replace(timetmrw.year, timetmrw.month, timetmrw.day, 17, 0, 0, tzinfo=currtime.tzinfo)
    #currtime.replace... - replaces the date and time components but doesn't change the timezone
    #tzinfo=currtime.tzinfo - sets time zone of tmrw5 to be the same as currtime
    return (tmrw5 - currtime).total_seconds() / 3600

In [14]:
descs = ["5pm UTC today to 5pm UTC tmrw", 
         "5pm PST today to 5pm PST tmrw",
         "5pm EST today to 5pm EST tmrw",
         "5pm CET today to 5pm CET tmrw",
         "5pm JST today to 5pm JST tmrw"]
#fold = 0 (pre dst) or 1 (post dst)
exps = [24, 24, 24, 24, 24]
currtimes = [
    datetime(2024, 6, 20, 17, 0, 0, tzinfo=timezone.utc),  # UTC
    datetime(2024, 6, 20, 17, 0, 0, tzinfo=pytz.timezone('US/Pacific')),  # PST
    datetime(2024, 6, 20, 17, 0, 0, tzinfo=pytz.timezone('US/Eastern')),  # EST
    datetime(2024, 6, 20, 17, 0, 0, tzinfo=pytz.timezone('CET')),  # CET
    datetime(2024, 6, 20, 17, 0, 0, tzinfo=pytz.timezone('Asia/Tokyo'))  # JST
]

#print("HELLO: ", time_until_5_tmrw(datetime(2024, 11, 3, hour=2, minute=0, second=0, fold=0)))
#print("HELLO: ", time_until_5_tmrw(datetime(2024, 11, 3, hour=1, minute=59, second=0, fold=0)))
#print("HELLO: ", time_until_5_tmrw(datetime(2024, 11, 3, hour=3, minute=0, second=0, fold=0)),"\n")
for i in range(len(descs)):
    print(descs[i])
    print("expected: ",exps[i]," hrs")
    #print("og: ", time_until_5_tmrw(currtimes[i])) - "can't subtract offset-naive and offset-aware datetimes"
    print("using timezone: ", time_until_5_tmrw_timezone(currtimes[i]))
    print("using pytz: ", time_until_5_tmrw_pytz(currtimes[i]))
    print("using dateutil.tz: ", time_until_5_tmrw_tz(currtimes[i]))
    print("using zoneinfo: ", time_until_5_tmrw_zoneinfo(currtimes[i]))
    print("\n")

5pm UTC today to 5pm UTC tmrw
expected:  24  hrs
using timezone:  24.0
using pytz:  24.0
using dateutil.tz:  24.0
using zoneinfo:  24.0


5pm PST today to 5pm PST tmrw
expected:  24  hrs
using timezone:  24.0
using pytz:  16.116666666666667
using dateutil.tz:  16.116666666666667
using zoneinfo:  24.0


5pm EST today to 5pm EST tmrw
expected:  24  hrs
using timezone:  24.0
using pytz:  19.066666666666666
using dateutil.tz:  19.066666666666666
using zoneinfo:  24.0


5pm CET today to 5pm CET tmrw
expected:  24  hrs
using timezone:  24.0
using pytz:  25.0
using dateutil.tz:  25.0
using zoneinfo:  24.0


5pm JST today to 5pm JST tmrw
expected:  24  hrs
using timezone:  24.0
using pytz:  33.31666666666667
using dateutil.tz:  33.31666666666667
using zoneinfo:  24.0




**localize** - attached timezone info to a naive datetime object
**normalize** - adjusts datetime objects to ensure they're in a valid state according to the rules of that timezone; handles transitions between standard time and daylight saving time